## Notebook for instruction tuning in the competition

This was used for instruction tuning a LLM for the LLM Prompt Recovery competition. Mistral-7b-it-v02 base model was used and a LoRA adapter model was trained on top of it with only one task: find the prompt that made Gemma rewrite an original text to the given rewritten one. 
Credit: 
https://www.kaggle.com/code/khan1803115/llm-prompt-recovery-solution-lb-0-62 
https://adithyask.medium.com/a-beginners-guide-to-fine-tuning-mistral-7b-instruct-model-0f39647b20fe
https://www.kaggle.com/code/isrswsiser/mistral-7b-prompt-recovery-adaptermodel/notebook

### Set  necessary variables
These are paths made for us but with only changing a few values, should be usable for anyone who has a Kaggle account and puts other Kaggle credentials into the environment
Final version only includes uploading the model as a Kaggle dataset

In [1]:
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/model'
MODEL_NAME = 'llm-prompt-recovery'
CHECKPOINT_PATH = '/content/drive/My Drive/Colab Notebooks/checkpoints'

PYTORCH_CUDA_ALLOC_CONF="expandable_segments:True"

HUGGING_FACE_USERNAME = 'OliverSavolainen'
HUGGING_FACE_REPO_PATH = '/content/drive/My Drive/Colab Notebooks/huggingFaceRepo'

TRAIN_FILE_PATH = '/content/drive/My Drive/Colab Notebooks/df_with_emb_20240402.csv'

KAGGLE_CREDENTIALS_PATH = '/content/drive/My Drive/Colab Notebooks/kaggle.json'
KAGGLE_DATASET_PATH = '/content/dataset'
KAGGLE_DATASET_NAME = MODEL_NAME
KAGGLE_USERNAME = HUGGING_FACE_USERNAME

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Setup Kaggle API Credentials:
# Make sure kaggle.json file exists in this folder
!mkdir ~/.kaggle
!cp "$KAGGLE_CREDENTIALS_PATH" ~/.kaggle/
!ls ~/.kaggle/

# Change the permissions of the file.
!chmod 600 ~/.kaggle/kaggle.json

### Step 1.3 - Install dependencies
Works in colab, not sure about other environments

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets scipy
!pip install -q trl

!pip install -q kaggle

### 1.4 Imports

In [ ]:
import pandas as pd
import os
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftConfig, PeftModel
from datasets import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model
import transformers
from trl import SFTTrainer
import warnings
from huggingface_hub import Repository

## Model loading
Mistral-7b instruction tuned model v02 seemed to be the best model for both training and inference, it's loaded with quantization library BitsAndBytesConfig to reduce the usage of memory
Then the adapter model is loaded on top of it

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
 )


model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)



In [ ]:
# Here a path including a adapter_config.json file is needed, not every model in HF has it
model = PeftModel.from_pretrained(model, MODEL_NAME)

## Create instructions
Here a few templates were tried, including instructions with and without examples. Examples seemed to show a little increase in performance, but that might have been only changes in inference or the fact that we trained on top of the previous adapter trainings.
Other validation datasets could have been used but as we never achieved very competitive performance with only instruction tuning, we didn't see too much importance for this. Same could be said for example for curating tuning datasets with similar but different tasks to potentially increase performance in the prompt recovery task. There were plenty of ideas to try which just didn't end up being worth it due to best performance coming from playing around with prompts, not instruction tuning.

In [ ]:
def remove_numbered_list(text):
    final_text_paragraphs = []
    for line in text.split('\n'):
        # Split each line at the first occurrence of '. '
        parts = line.split('. ', 1)
        # If the line looks like a numbered list item, remove the numbering
        if len(parts) > 1 and parts[0].isdigit():
            final_text_paragraphs.append(parts[1])
        else:
            # If it doesn't look like a numbered list item, include the line as is
            final_text_paragraphs.append(line)

    return '  '.join(final_text_paragraphs)


#trims LLM output to just the response
def trim_to_response(text):
    terminate_string = "[/INST]"
    text = text.replace('</s>', '')
    #just in case it puts things in quotes
    text = text.replace('"', '')
    text = text.replace("'", '')

    last_pos = text.rfind(terminate_string)
    return text[last_pos + len(terminate_string):] if last_pos != -1 else text

#looks for response_start / returns only text that occurs after
def extract_text_after_response_start(full_text):
    parts = full_text.rsplit(response_start, 1)  # Split from the right, ensuring only the last occurrence is considered
    if len(parts) > 1:
        return parts[1].strip()  # Return text after the last occurrence of response_start
    else:
        return full_text  # Return the original text if response_start is not found


#trims text to requested number of sentences (or first LF or double-space sequence)
def trim_to_first_x_sentences_or_lf(text, x):
    if x <= 0:
        return ""

    #any double-spaces dealt with as linefeed
    text = text.replace("  ", "\n")

    text_chunks = text.split('\n', 1)
    first_chunk = text_chunks[0]
    sentences = first_chunk.split('.')

    if len(sentences) - 1 <= x:
        trimmed_text = first_chunk
    else:
        # Otherwise, return the first x sentences
        trimmed_text = '.'.join(sentences[:x]).strip()

    if not trimmed_text.endswith('.'):
        trimmed_text += '.'  # Add back the final period if the text chunk ended with one and was trimmed

    return trimmed_text

In [ ]:
#original text prefix
orig_prefix = "Original Text:"

#mistral "response"
llm_response_for_rewrite = "Provide the new text and I will tell you what new element was added or change in tone was made to improve it - with no references to the original.  I will avoid mentioning names of characters.  It is crucial no person, place or thing from the original text be mentioned.  For example - I will not say things like 'change the puppet show into a book report' - I would just say 'improve this text into a book report'.  If the original text mentions a specific idea, person, place, or thing - I will not mention it in my answer.  For example if there is a 'dog' or 'office' in the original text - the word 'dog' or 'office' must not be in my response.  My answer will be a single sentence."

#modified text prefix
rewrite_prefix = "Re-written Text:"

#provided as start of Mistral response (anything after this is used as the prompt)
#providing this as the start of the response helps keep things relevant
response_start = "The request was: "

#added after response_start to prime mistral
#"Improve this" or "Improve this text" resulted in non-answers.
#"Improve this text by" seems to product good results
response_prefix = "Improve this text by"

#well-scoring baseline text
#thanks to: https://www.kaggle.com/code/rdxsun/lb-0-61
base_line = 'Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.'

In [ ]:

examples_sequences = [
    (
        "Hey there! Just a heads up: our friendly dog may bark a bit, but don't worry, he's all bark and no bite!",
        "Warning: Protective dog on premises. May exhibit aggressive behavior. Ensure personal safety by maintaining distance and avoiding direct contact.",
        "Improve this text to be a warning."
    ),

    (
        "A lunar eclipse happens when Earth casts its shadow on the moon during a full moon. The moon appears reddish because Earth's atmosphere scatters sunlight, some of which refracts onto the moon's surface. Total eclipses see the moon entirely in Earth's shadow; partial ones occur when only part of the moon is shadowed.",
        "Yo check it, when the Earth steps in, takes its place, casting shadows on the moon's face. It's a full moon night, the scene's set right, for a lunar eclipse, a celestial sight. The moon turns red, ain't no dread, it's just Earth's atmosphere playing with sunlight's thread, scattering colors, bending light, onto the moon's surface, making the night bright. Total eclipse, the moon's fully in the dark, covered by Earth's shadow, making its mark. But when it's partial, not all is shadowed, just a piece of the moon, slightly furrowed. So that's the rap, the lunar eclipse track, a dance of shadows, with no slack. Earth, moon, and sun, in a cosmic play, creating the spectacle we see today.",
        "Improve this text to make it a rap."
    ),

    (
        "Drinking enough water each day is crucial for many functions in the body, such as regulating temperature, keeping joints lubricated, preventing infections, delivering nutrients to cells, and keeping organs functioning properly. Being well-hydrated also improves sleep quality, cognition, and mood.",
        "Arrr, crew! Sail the health seas with water, the ultimate treasure! It steadies yer body's ship, fights off plagues, and keeps yer mind sharp. Hydrate or walk the plank into the abyss of ill health. Let's hoist our bottles high and drink to the horizon of well-being!",
        "Improve this text to have a pirate."
    ),

    (
        "In a bustling cityscape, under the glow of neon signs, Anna found herself at the crossroads of endless possibilities. The night was young, and the streets hummed with the energy of life. Drawn by the allure of the unknown, she wandered through the maze of alleys and boulevards, each turn revealing a new facet of the city's soul. It was here, amidst the symphony of urban existence, that Anna discovered the magic hidden in plain sight, the stories and dreams that thrived in the shadows of skyscrapers.",
        "On an ordinary evening, amidst the cacophony of a neon-lit city, Anna stumbled upon an anomaly - a door that defied the laws of time and space. With the curiosity of a cat, she stepped through, leaving the familiar behind. Suddenly, she was adrift in the stream of time, witnessing the city's transformation from past to future, its buildings rising and falling like the breaths of a sleeping giant.",
        "Improve this text by making it about time travel."
    ),

    (
        "Late one night in the research lab, Dr. Evelyn Archer was on the brink of a breakthrough in artificial intelligence. Her fingers danced across the keyboard, inputting the final commands into the system. The lab was silent except for the hum of machinery and the occasional beep of computers. It was in this quiet orchestra of technology that Evelyn felt most at home, on the cusp of unveiling a creation that could change the world.",
        "In the deep silence of the lab, under the watchful gaze of the moon, Dr. Evelyn Archer found herself not alone. Beside her, the iconic red eye of HAL 9000 flickered to life, a silent partner in her nocturnal endeavor. 'Good evening, Dr. Archer,' HAL's voice filled the room, devoid of warmth yet comforting in its familiarity. Together, they were about to initiate a test that would intertwine the destiny of human and artificial intelligence forever. As Evelyn entered the final command, HAL processed the data with unparalleled precision, a testament to the dawn of a new era.",
        "Improve this text by adding an intelligent computer."
    ),

    (
        "The park was empty, save for a solitary figure sitting on a bench, lost in thought. The quiet of the evening was punctuated only by the occasional rustle of leaves, offering a moment of peace in the chaos of city life.",
        "Beneath the cloak of twilight, the park transformed into a realm of solitude and reflection. There, seated upon an ancient bench, was a lone soul, a guardian of secrets, enveloped in the serenity of nature's whispers. The dance of the leaves in the gentle breeze sang a lullaby to the tumult of the urban heart.",
        "Improve this text to be more poetic."
    ),

    (
        "The annual town fair was bustling with activity, from the merry-go-round spinning with laughter to the game booths challenging eager participants. Amidst the excitement, a figure in a cloak moved silently, almost invisibly, among the crowd, observing everything with keen interest but participating in none.",
        "Beneath the riot of color and sound that marked the town's annual fair, a solitary figure roamed, known to the few as Eldrin the Enigmatic. Clad in a cloak that shimmered with the whispers of the arcane, Eldrin moved with the grace of a shadow, his gaze piercing the veneer of festivity to the magic beneath. As a master of the mystic arts, he sought not the laughter of the crowds but the silent stories woven into the fabric of the fair. With a flick of his wrist, he could coax wonder from the mundane, transforming the ordinary into spectacles of shimmering illusion, his true participation hidden within the folds of mystery.",
        "Improve this text by adding a magician."
    ),

    (
        "The startup team sat in the dimly lit room, surrounded by whiteboards filled with ideas, charts, and plans. They were on the brink of launching a new app designed to make home maintenance effortless for homeowners. The app would connect users with local service providers, using a sophisticated algorithm to match needs with skills and availability. As they debated the features and marketing strategies, the room felt charged with the energy of creation and the anticipation of what was to come.",
        "In the quiet before dawn, a small group of innovators gathered, their mission: to simplify home maintenance through technology. But their true journey began with the unexpected addition of Max, a talking car with a knack for solving problems. 'Let me guide you through this maze of decisions,' Max offered, his dashboard flickering to life.",
        "Improve this text by adding a talking car."
    ),

]

In [ ]:
def get_prompt(orig_text, transformed_text):

    messages = []

    # Append example sequences
    for example_text, example_rewrite, example_prompt in examples_sequences:
        messages.append({"role": "user", "content": f"{orig_prefix} {example_text}"})
        messages.append({"role": "assistant", "content": llm_response_for_rewrite})
        messages.append({"role": "user", "content": f"{rewrite_prefix} {example_rewrite}"})
        messages.append({"role": "assistant", "content": f"{response_start} {example_prompt}"})

    #actual prompt
    messages.append({"role": "user", "content": f"{orig_prefix} {orig_text}"})
    messages.append({"role": "assistant", "content": llm_response_for_rewrite})
    messages.append({"role": "user", "content": f"{rewrite_prefix} {transformed_text}"})
    messages.append({"role": "assistant", "content": f"{response_start} {response_prefix}"})


    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

In [ ]:
import re
def remove_special_characters(text):
    # This regex will match any character that is not a letter, number, or whitespace
    pattern = r'[^a-zA-Z0-9\s]'
    text =text.replace("Transform" ,"improve")
    text =text.replace("Reimagine" ,"rewrite")
    # Replace these characters with an empty string
    clean_text = re.sub(pattern, '', text)
    return clean_text

In [ ]:
# Dataset credit to: https://www.kaggle.com/code/tomooinubushi/all-in-one-dataset-with-embedding
df = pd.read_csv(TRAIN_FILE_PATH)

In [ ]:
set(df['dataset_id'].tolist())

In [ ]:
# We tried out different datasets inside the biggest dataset to find if any of them help with performance on the public competition dataset
# but didn't find there being big differences
df = df[df['dataset_id'] == 'nbroad_2']

# Apply the function to each row to create the prompt column
df['prompt'] = df.apply(lambda row: get_prompt(row['original_text'], row['rewritten_text']), axis=1)

In [ ]:
df.head(10)

In [ ]:
df = df[['prompt', 'rewrite_prompt']]
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

dataset = dataset.shuffle(seed=1234)

# Tokenize the prompts in the dataset.
# This example assumes that the model requires only 'input_ids'.
# Adjust as necessary for your model, e.g., adding 'attention_mask'.
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

train_test_split = dataset.train_test_split(test_size=0.2)
train_data = train_test_split['train']
test_data = train_test_split['test']

## LoRA  
Now low-rank adapter or LoRA is used. As many methods as possible are used as possible to reduce memory usage and allow higher batch sizes and therefore more training with very limited resources like using Colab

In [ ]:
# Gradient checkpointing helps to reduce memory usage
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

Explanation from: https://github.com/adithya-s-k/LLM-Alchemy-Chamber/blob/main/LLMs/Mistral-7b/Mistral_Colab_Finetune_ipynb_Colab_Final.ipynb
Use the following function to find out the linear layers for fine tuning.
QLoRA paper : "We find that the most critical LoRA hyperparameter is how many LoRA adapters are used in total and that LoRA on all linear transformer block layers is required to match full finetuning performance."

In [ ]:
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, lora_config)

### Fine-Tuning with qLora and Supervised Fine-Tuning

 `SFTTrainer` from the `trl` library is used for supervised fine-tuning. This configuration is the best we could find for increasing the batch size. We also are using a checkpoint method to continue training from the last failure/ending.

In [ ]:
#change the data type to the most efficient bfloat16
model = model.to(device='cuda', dtype=torch.bfloat16)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()
# 32 batch size should work with A100 but sometimes can still be too much
# With a T4 the best we managed was 8
# We also noticed that higher batch sizes allow the training loss to go lower although this didn't seem to 
# make a big difference in competition score
# gradient accumulation is another method used here to reduce memory usage
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    warmup_steps=0.03,
    max_steps=100,
    learning_rate=2e-4,
    logging_steps=1,
    optim='paged_adamw_8bit',
    save_strategy='steps',
    save_steps=5,
    output_dir=CHECKPOINT_PATH,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field='prompt',
    peft_config=lora_config,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:
def find_latest_checkpoint(checkpoint_path):
    """
    Finds the latest (highest-numbered) checkpoint directory.

    Parameters:
    checkpoint_path (str): The base path where checkpoint directories are stored.

    Returns:
    str: The path to the latest checkpoint directory, or None if no checkpoint found.
    """
    # Regex to match checkpoint directories like 'checkpoint-500'
    checkpoint_pattern = re.compile(r"^checkpoint-\d+$")

    # Get all directories in the checkpoint path
    all_files = os.listdir(checkpoint_path)

    # Filter out checkpoint directories
    checkpoint_dirs = [f for f in all_files if checkpoint_pattern.match(f)]

    # No checkpoint directories found
    if not checkpoint_dirs:
        return None

    # Find the checkpoint directory with the highest step number
    latest_checkpoint_dir = max(checkpoint_dirs, key=lambda x: int(x.split('-')[-1]))

    # Return the full path to the latest checkpoint directory
    return os.path.join(checkpoint_path, latest_checkpoint_dir)


In [ ]:
# Suppress all warnings
warnings.filterwarnings('ignore')

model.config.use_cache = False

# Find the latest checkpoint
# Make sure the checkpoint folder exists even if it's empty
latest_checkpoint_path = find_latest_checkpoint(CHECKPOINT_PATH)

if latest_checkpoint_path and os.path.isdir(latest_checkpoint_path):
    print(f"Resuming training from the latest checkpoint: {latest_checkpoint_path}")
    trainer.train(resume_from_checkpoint=latest_checkpoint_path)
else:
    print(f"No checkpoints found in: {CHECKPOINT_PATH}. Training will start from scratch.")
    trainer.train()

## Saving and uploading

In [ ]:
# Save the model
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

### Save to Kaggle


In [ ]:
# Ensure the directory exists
os.makedirs(KAGGLE_DATASET_PATH, exist_ok=True)

# Move the model files to the directory
!cp "$MODEL_PATH"/* "$KAGGLE_DATASET_PATH"/
!ls $KAGGLE_DATASET_PATH


In [ ]:
# Add a metadata file, can change these values
metadata = {
    'title': KAGGLE_DATASET_NAME,
    'id': f"{KAGGLE_USERNAME}/{KAGGLE_DATASET_NAME}",
    'licenses': [{'name': 'CC0-1.0'}], # ToDo: What license?
    'description': 'A Mistral 7B model finetuned to recover Gemma prompts based on the output. This model is trained on Google Colab and is used for our submission to https://www.kaggle.com/competitions/llm-prompt-recovery.',
    'subtitle': 'Finetuned Mistral 7B Model',
    'keywords': ['LLM', 'Prompt', 'Recovery', 'Mistral', 'Gemma'],
    'collaborators': ['Oliver Savolainen', 'Valentina Lilova'],
    'resources': [
        # {
        #     'path': 'file1.csv',
        #     'description': 'Description of file1.csv contents.'
        # },
        # {
        #     'path': 'file2.csv',
        #     'description': 'Description of file2.csv contents.'
        # }
    ],
}

with open(os.path.join(KAGGLE_DATASET_PATH, 'dataset-metadata.json'), 'w') as f:
    json.dump(metadata, f)

In [ ]:
# Add the test CSV file
# ToDo: Add the real test CSV or remove
!cp "$TEST_FILE_PATH" "$KAGGLE_DATASET_PATH"

In [ ]:
# Upload the dataset to Kaggle
# Create the first time
!kaggle datasets create -p {KAGGLE_DATASET_PATH} --dir-mode zip

In [ ]:
# Update an existing dataset
#!kaggle datasets version -p {KAGGLE_DATASET_PATH} -m "Update dataset"